<a href="https://colab.research.google.com/github/rodrigoms95/herramientas-climatico-22-1/blob/main/code/Proyecto/Jupyter/ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
# Entrena un modelo de Gradient Boosted Trees
# para predecir el consumo eléctrico
# y la cantidad de usuarios.

# Tomado y modificado a partir de:
# https://www.datacamp.com/community/tutorials/xgboost-in-python
# https://towardsdatascience.com/a-beginners-guide-to-xgboost-87f5d4c30ed7

import pandas as pd
import numpy as np

import xgboost as xgb

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [85]:
path_drive = "/content/drive/MyDrive/Colab/" 
path_data = path_drive + "data/"
fname = "data.csv"

data = pd.read_csv(path_data + fname)

# Información utilizada para el modelo.
data.head()

,NOM_ENT,NOM_MUN,Consumo_1*,Usuarios_1*,lon,lat,T_mean,CDD_mean,Pre,Pre_Tmean,Densidad_población,PCI,$GLP,Población,PIB,Año,CVE_INEGI
0,Aguascalientes,Aguascalientes,302560818.0,235978.0,-102.295872,21.811400,19.460571,517.162109,450.955782,400.161505,6.833315,146798.659992,10.151,797010.0,1.170000e+11,2010,1001
1,Aguascalientes,Asientos,12805524.0,10624.0,-102.045590,22.126479,18.816061,562.153564,349.574747,302.348158,0.837213,65304.649499,10.151,45492.0,2.970839e+09,2010,1002
2,Aguascalientes,Calvillo,17578788.0,15496.0,-102.704911,21.900640,19.020189,464.146240,513.303061,446.077944,0.586295,68511.043575,10.151,54136.0,3.708914e+09,2010,1003
3,Aguascalientes,Cosío,4015852.0,3529.0,-102.297038,22.360619,18.941561,577.571777,370.966605,316.695232,1.171645,82001.610624,10.151,15042.0,1.233468e+09,2010,1004
4,Aguascalientes,Jesús María,31208047.0,26103.0,-102.445700,21.932112,19.160786,487.516479,435.141371,378.999951,1.992218,112650.094989,10.151,99590.0,1.121882e+10,2010,1005


In [86]:
# Medias.
data.iloc[:,2:].mean(axis = 0)

Consumo_1*            2.156270e+07
Usuarios_1*           1.345679e+04
lon                  -9.874304e+01
lat                   2.002281e+01
T_mean                2.242712e+01
CDD_mean              4.964348e+02
Pre                   1.106790e+03
Pre_Tmean             8.369053e+02
Densidad_población    2.879387e+00
PCI                   8.464168e+04
$GLP                  1.099316e+01
Población             4.726342e+04
PIB                   6.765280e+09
Año                   2.013000e+03
CVE_INEGI             1.932416e+04
dtype: float64

In [87]:
# Desviaciones estándar.
data.iloc[:,2:].std(axis = 0)

Consumo_1*            8.631740e+07
Usuarios_1*           3.961645e+04
lon                   4.384691e+00
lat                   3.345834e+00
T_mean                3.377748e+00
CDD_mean              2.178876e+02
Pre                   6.336191e+02
Pre_Tmean             4.680760e+02
Densidad_población    1.183431e+01
PCI                   7.166418e+04
$GLP                  8.511531e-01
Población             1.360173e+05
PIB                   2.694315e+10
Año                   2.000058e+00
CVE_INEGI             7.381611e+03
dtype: float64

In [88]:
# Cuartiles.
data.iloc[:,2:].quantile([0.25, 0.5, 0.75], axis = 0)

,Consumo_1*,Usuarios_1*,lon,lat,T_mean,CDD_mean,Pre,Pre_Tmean,Densidad_población,PCI,$GLP,Población,PIB,Año,CVE_INEGI
0.25,1043139.5,1163.0,-100.737771,17.620009,19.967443,358.526184,682.422936,526.375082,0.191171,43039.532830,10.238682,4274.0,2.403014e+08,2011.0,14079.0
0.50,3224329.0,3308.0,-98.236805,19.329310,21.914389,434.459412,941.986782,723.676950,0.527785,66532.781697,11.104057,13017.0,8.416350e+08,2013.0,20226.0
0.75,10084256.0,8869.0,-96.769306,20.934726,24.924725,545.298798,1387.195159,1019.995059,1.374554,97519.163463,11.637004,33620.5,2.806443e+09,2015.0,24027.0


In [89]:
# Matriz de correlación de Pearson.
data.iloc[:,2:].corr()

,Consumo_1*,Usuarios_1*,lon,lat,T_mean,CDD_mean,Pre,Pre_Tmean,Densidad_población,PCI,$GLP,Población,PIB,Año,CVE_INEGI
Consumo_1*,1.000000,0.844769,-1.517104e-01,1.966613e-01,0.037609,0.175000,-0.108148,-0.093338,0.318539,0.280889,0.005908,0.807671,0.749397,1.314590e-02,-6.607402e-02
Usuarios_1*,0.844769,1.000000,-1.204962e-01,1.351577e-01,-0.047144,0.095912,-0.112452,-0.082853,0.541620,0.309517,0.017101,0.985200,0.889541,1.865308e-02,-1.208885e-01
lon,-0.151710,-0.120496,1.000000e+00,-6.648263e-01,0.368563,-0.570965,0.487079,0.412035,-0.000243,-0.270591,-0.013115,-0.111694,-0.085190,5.813482e-18,2.369749e-01
lat,0.196661,0.135158,-6.648263e-01,1.000000e+00,-0.172404,0.901895,-0.477210,-0.406219,-0.029756,0.368796,0.037059,0.121039,0.110049,1.078267e-17,-1.816479e-02
T_mean,0.037609,-0.047144,3.685626e-01,-1.724041e-01,1.000000,0.002264,0.406738,0.400219,-0.174850,-0.001128,0.025122,-0.066071,-0.048065,4.720407e-02,1.749890e-01
CDD_mean,0.175000,0.095912,-5.709655e-01,9.018954e-01,0.002264,1.000000,-0.402018,-0.337884,-0.071880,0.331921,0.036149,0.078861,0.075153,7.924268e-02,-4.882388e-02
Pre,-0.108148,-0.112452,4.870794e-01,-4.772099e-01,0.406738,-0.402018,1.000000,0.921505,-0.071052,-0.257785,-0.093505,-0.105271,-0.096163,-6.036922e-02,1.105634e-01
Pre_Tmean,-0.093338,-0.082853,4.120352e-01,-4.062194e-01,0.400219,-0.337884,0.921505,1.000000,-0.049735,-0.196991,-0.037105,-0.075536,-0.076816,-1.245438e-03,1.161193e-01
Densidad_población,0.318539,0.541620,-2.431198e-04,-2.975554e-02,-0.174850,-0.071880,-0.071052,-0.049735,1.000000,0.229053,0.003081,0.560745,0.585140,5.724501e-03,-7.796424e-02
PCI,0.280889,0.309517,-2.705908e-01,3.687962e-01,-0.001128,0.331921,-0.257785,-0.196991,0.229053,1.000000,0.066342,0.283659,0.473212,4.605540e-02,-1.489635e-01


# Cálculo sin logaritmos

In [90]:
# Escogemos el conjunto de features y de variables a predecir.
# Escogemos solo uno de los conjuntos a predecir.
i = 1
X, Y = data.iloc[:,6:], data.iloc[:,i + 1]

# Obtenemos los logaritmos de las variables.
log = False
if log:
  lncol = ["Consumo_1*", "Usuarios_1*", "Población", "PIB"]
  data[lncol] = np.log(data[lncol])

# Quitamos algunas features.
#X.drop("CVE_INEGI", axis = 1, inplace = True)
#X.drop(["lon", "lat"], axis = 1, inplace = True)

# Separamos en conjuntos de entrenamiento y de prueba.
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size = 0.2, random_state = 123 )

# Features a utilizar.
X.head()

,T_mean,CDD_mean,Pre,Pre_Tmean,Densidad_población,PCI,$GLP,Población,PIB,Año,CVE_INEGI
0,19.460571,517.162109,450.955782,400.161505,6.833315,146798.659992,10.151,797010.0,1.170000e+11,2010,1001
1,18.816061,562.153564,349.574747,302.348158,0.837213,65304.649499,10.151,45492.0,2.970839e+09,2010,1002
2,19.020189,464.146240,513.303061,446.077944,0.586295,68511.043575,10.151,54136.0,3.708914e+09,2010,1003
3,18.941561,577.571777,370.966605,316.695232,1.171645,82001.610624,10.151,15042.0,1.233468e+09,2010,1004
4,19.160786,487.516479,435.141371,378.999951,1.992218,112650.094989,10.151,99590.0,1.121882e+10,2010,1005


In [91]:
# Variable a predecir.
Y.head()

0    302560818.0
1     12805524.0
2     17578788.0
3      4015852.0
4     31208047.0
Name: Consumo_1*, dtype: float64

In [92]:
# Método 0.
# Regresión lineal.

# Creamos el regresor.
lin_reg = LinearRegression()

# Entrenamos el modelo.
lin_reg.fit(X_train, Y_train)

# Evaluamos el modelo.
preds = lin_reg.predict(X_test)

# Calculamos el error.
if log:
    rmse = np.sqrt(mean_squared_error(
        np.exp(Y_test), np.exp(preds)))
else:
    rmse = np.sqrt(mean_squared_error(Y_test, preds))
print(f"RMSE: {rmse:.3E}")

RMSE: 4.356E+07


In [93]:
# Método 1.
# Entrenamiento simple.

# Hiperparámetros.
params = {
    "objective": "reg:squarederror",
    "colsample_bytree": 0.3,
    "learning_rate": 0.1,
    "max_depth": 50,
    #"min_child_weight" : 5,
    "alpha": 10,
    #"gamma": 0.2,
    "n_estimators": 100,
    "tree_method": "gpu_hist"
    }

# Creamos el regresor con los hiperparámetros.
xg_reg = xgb.XGBRegressor( **params )

# Entrenamos el modelo.
xg_reg.fit(X_train, Y_train, verbose = True)

# Evaluamos el modelo.
preds = xg_reg.predict(X_test)

# Calculamos el error.
if log:
    rmse = np.sqrt(mean_squared_error(
        np.exp(Y_test), np.exp(preds)))
else:
    rmse = np.sqrt(mean_squared_error(Y_test, preds))
print(f"RMSE: {rmse:.3E}")

RMSE: 2.385E+07


# Cálculo con logaritmos

In [94]:
# Obtenemos los logaritmos de las variables.
log = True
if log:
  lncol = ["Consumo_1*", "Usuarios_1*", "Población", "PIB"]
  data[lncol] = np.log(data[lncol])

# Escogemos el conjunto de features y de variables a predecir.
# Escogemos solo uno de los conjuntos a predecir.
i = 1
X, Y = data.iloc[:,6:], data.iloc[:,i + 1]

# Quitamos algunas features.
#X.drop("CVE_INEGI", axis = 1, inplace = True)
#X.drop(["lon", "lat"], axis = 1, inplace = True)

# Separamos en conjuntos de entrenamiento y de prueba.
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size = 0.2, random_state = 123 )

In [95]:
# Método 0.
# Regresión lineal.

# Creamos el regresor.
lin_reg = LinearRegression()

# Entrenamos el modelo.
lin_reg.fit(X_train, Y_train)

# Evaluamos el modelo.
preds = lin_reg.predict(X_test)

# Calculamos el error.
if log:
    rmse = np.sqrt(mean_squared_error(
        np.exp(Y_test), np.exp(preds)))
else:
    rmse = np.sqrt(mean_squared_error(Y_test, preds))
print(f"RMSE: {rmse:.3E}")

RMSE: 3.636E+07


In [96]:
# Método 1.
# Entrenamiento simple.

# Hiperparámetros.
params = {
    "objective": "reg:squarederror",
    "colsample_bytree": 0.3,
    "learning_rate": 0.1,
    "max_depth": 50,
    #"min_child_weight" : 5,
    "alpha": 10,
    #"gamma": 0.2,
    "n_estimators": 100,
    "tree_method": "gpu_hist"
    }

# Creamos el regresor con los hiperparámetros.
xg_reg = xgb.XGBRegressor( **params )

# Entrenamos el modelo.
xg_reg.fit(X_train, Y_train, verbose = True)

# Evaluamos el modelo.
preds = xg_reg.predict(X_test)

# Calculamos el error.
if log:
    rmse = np.sqrt(mean_squared_error(
        np.exp(Y_test), np.exp(preds)))
else:
    rmse = np.sqrt(mean_squared_error(Y_test, preds))
print(f"RMSE: {rmse:.3E}")

RMSE: 2.340E+07


# Otras formas de entrenar el modelo

Primero hacer funcionar la principal estrategia.

In [ ]:
# Escogemos el conjunto de features y de variables a predecir.
# Escogemos solo uno de los conjuntos a predecir.
i = 1
X, Y = data.iloc[:,6:], data.iloc[:,i + 1]

# Obtenemos los logaritmos de las variables.
log = False
if log:
  lncol = ["Consumo_1*", "Usuarios_1*", "Población", "PIB"]
  data[lncol] = np.log(data[lncol])

# Quitamos algunas features.
#X.drop("CVE_INEGI", axis = 1, inplace = True)
#X.drop(["lon", "lat"], axis = 1, inplace = True)

# Separamos en conjuntos de entrenamiento y de prueba.
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size = 0.2, random_state = 123 )

# Features a utilizar.
X.head()

In [ ]:
# Método 2.
# Ajuste de hiperparámetros.

# Hiperparámetros.
params = {
    "colsample_bytree": [ 0.3, 0.4, 0.5 , 0.7 ],
    "learning_rate": [0.05, 0.10, 0.15, 0.20, 0.25, 0.30],
    "max_depth": [10, 20, 30, 40, 50, 60, 70],
    #"min_child_weight" : [ 1, 3, 5, 7 ],
    "alpha": [7, 8, 9, 10, 11, 12, 13],
    #"gamma": [0.0, 0.1, 0.2 , 0.3, 0.4],
    "n_estimators": [50, 100, 200, 300, 400, 500, 700],
    }

# Creamos el regresor.
xg_reg = xgb.XGBRegressor(
    objective = "reg:squarederror",
    tree_method = "gpu_hist" )

# Ajustador de hiperparámetros con Cross Validation.
grid = GridSearchCV(xg_reg, params,
    n_jobs = -1, cv = 5, verbose = 2
    scoring = "neg_mean_absolute_error")

# Entrenamos el modelo.
grid.fit(X_train, Y_train)

# Evaluamos el modelo.
preds = grid.predict(X_test)

# Calculamos el error.
rmse = np.sqrt(mean_squared_error(Y_test, preds))
print("RMSE: %f" % (rmse))

# Escoge las mejores predicciones ??????
# ¿¿Por qué??
best_preds = np.asarray([np.argmax(line) for line in preds])

# Más medidas para evaluar el modelo.
print("Precision = {}".format(precision_score(Y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(Y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(Y_test, best_preds)))

In [12]:
# Método 3.
# Cross Validation con XGBoost.

# Hiperparámetros.
params = {
    "objective": "reg:squarederror",
    "colsample_bytree": 0.3,
    "learning_rate": 0.1,
    "max_depth": 10,
    #"min_child_weight" : 5,
    "alpha": 10,
    #"gamma": 0.2,
    "n_estimators": 100,
    "tree_method": "gpu_hist"
    }

# Ajustamos el DataFrame al formato de XGBoost.
data_dmatrix = xgb.DMatrix(data = X, label = Y)

# Realizamos el entrenamiento con Cross Validation.
cv_results = xgb.cv(dtrain = data_dmatrix, params = params, nfold = 3,
    num_boost_round = 50, early_stopping_rounds = 10, metrics = "rmse",
    as_pandas = True, seed = 123)

print((cv_results["test-rmse-mean"]).tail(1))

49    0.361977
Name: test-rmse-mean, dtype: float64
